# Metapredicados y predicados de control

En Prolog tenemos una serie de herramientas muy poderosas y útiles ya definidas en la propia librería interna de Prolog, una serie de "funciones" intrínsecas del lenguaje como sucede con por ejemplo Java. Las podemos dividir entre **metapredicados**(No, no son accionistas de Zuckerberg, se llaman **meta** porque sus argumentos son otros predicados. Vamos, [predicados dentro de otros predicados](https://www.youtube.com/watch?v=SsShSms06-8)), **predicados de control** para influenciar el flujo que seguirá el motor de inferencia(Ya has visto en capítulos anteriores el uso del **"No se puede demostrar"**) o los predicados sobre la base de hechos(Que nos permiten añadir o eliminar hechos). Hablaremos ahora de este tipo de predicados especiales y cómo deberían utilizarse.<br>
Todos ellos se denotan de la manera **"predicado/número de argumentos"**. <br>
Vamos a ver los más importantes y **por qué** deberías acordarte de ellos como del cumpleaños de tu gato.<br>

## Metapredicados

### call/1 - El buen procastineo.
El predicado **call/1** nos permite, sencillamente, llamar a un predicado en concreto. Si bien puede parecer redundante, nos permite poder gestionar las llamadas a predicados de manera dinámica. Es útil cuando queremos ejecutar un predicado que desconocemos en tiempo de compilación pero se determina en tiempo de compilación.<br>
Vamos, que si no lo sabemos **ahora** no es un problema, ya se encarga el motor de inferencia de hacer nuestro trabajo:<br>
    
    choose_operation(Op) :-
    ( Op = add -> call(add, 2, 3);
      Op = multiply -> call(multiply, 2, 3) ).
En este ejemplo, la operación a realizar dependerá de la operación(El valor con el que se haya ligado la variable "Op").<br>

También resulta útil desde un punto de vista de buenas prácticas, permitiendo escribir código reutilizable para distintos predicados, resultando también una programación de más alto nivel, más flexible y comprensible desde nuestro punto de vista: <br>

    apply_to_all(Predicate, List) :-
        maplist(call(Predicate), List).

Este predicado aplicará el predicado del primer argumento a la lista del segundo argumento. ¿Cómo? **Buena pregunta**, vamos con el **maplist**.
### maplist/2-5 - Trabaja con cabeza, no a cabezazos.
Este predicado aplica un predicado a todos los elementos de entre una y cuatro listas, dependiendo de nuestras necesidades y la aridad de la cabecera.<br>
Para un predicado y una lista, la aridad de maplist será dos(maplist/2), para dos listas la aridad será tres, tres listas aridad cuatro, cuatro listas aridad cinco...<br> 

    es_par(X) :- 0 is X mod 2.
    
    todos_pares(List) :-
        maplist(es_par, List).
"todos_pares" aplica el predicado "es_par" a todos los elementos de una lista, sin tener que recorrer la lista manualmente dividiéndola entre cabeza y cola (\[H|T]). Esto es algo que todavía no has visto y no entenderás pero confía en mi, es **Bastante** más cómodo.<br>

### include/3 y exclude/3 - [Filtro, filtro, que es lo que les j...](https://youtu.be/ZttMiaF8PDI?si=OSiPxtpCj2Jr3IUI&t=8)
Estos dos predicados nos permiten filtrar listas basándonos en una condición, un predicado que recibe como **primer** parámetro. **Include** nos generará una lista con todos los elementos que cumplan la condición especificada, mientras que **exclude** lo hará con aquellos elementos que **no** la cumplan.<br>

    include(Predicate, List, FilteredList)./exclude(Predicate, List, FilteredList).
El primer argumento será el predicado que determina la condición, el segundo la lista a la que queremos aplicar la condición y el tercer argumento será una variable que unificará con todos los elementos de la lista que cumplan la condición.

### findall/3, bagof/3 y setof/3 - [Soy prologramador, deme deme deme.](https://youtu.be/FXbVANBSs-Y?si=8MNhjdkEqzn4dkaX&t=6)
Ojo aquí. Esto es ya plutonio enriquecido. Estos tres predicados son literalmente la santa Trinidad de la metaprogramación en Prolog, tan pronto te curan la ceguera como te tiran cuatro meteoritos en tu casa, te inundan el váter y te convierten en una estatua de sal ahumada.<br>
El primer predicado, **findall** tiene tres argumentos, el elemento que queremos recoger, el predicado que ha de cumplir y la lista donde se almacenará el resultado:<br>
    
    findall(Template, Goal, List).
**Template** es el elemento(Un **término**, y recuerda que un **término** puede ser CASI cualquier cosa) que queremos que se cumpla en **Goal** y se recoja en **List**.<br>
Por ejemplo:<br>
    
    menor_que_cinco(X) :- X < 5.

    recoge_numeros(List) :-
    findall(X, menor_que_cinco(X), List).
Queremos los elementos X que sean verdaderos en "menor_que_cinco(X)". "findall" nos devolverá todas las soluciones que cumplan esto **sin** ordenar, **sin filtrar** duplicados ni hacer nada más. Para la lista \[3,1,2,5,1,7,8], findall devolverá \[3,1,2,1], mientras que para la lista \[6,8,6,5,8] devolverá [].<br>

El segundo predicado, **bagof**, es **exactamente** igual que findall, **PERO** en donde **findall** devolvería una lista vacía si no encuentra nada, **bagof** fallará si no encuentra al menos una solución. Al igual que findall, **no** ordena la solución y **no** elimina duplicados.<br>

    menor_que_cinco(X) :- X < 5.

    recoge_numeros(List) :-
    bagof(X, menor_que_cinco(X), List).
Para la lista \[3,1,2,5,1,7,8], findall devolverá \[3,1,2,1], mientras que para la lista \[6,8,6,5,8] devolverá **false**.<br>

El tercer predicado, **setof**, tiene la misma sintaxis que sus dos hermanos más brutos, pero como lo criaron en una casa estable en lugar de entre una manada de lobos y hienas es un poco más refinado y sensato. En lugar de devolverte la solución de cualquier manera a la que te jode, **setof** ordenará primorosamente la solución para facilitarte el trabajo y **además** se tomará la molestia de elaborar una lista de soluciones óptima, **eliminando** las soluciones duplicadas y, por tanto, redundantes. Todo un fenómeno.<br>

    menor_que_cinco(X) :- X < 5.

    recoge_numeros(List) :-
    setof(X, menor_que_cinco(X), List).
Para la lista \[3,1,2,5,1,7,8], findall devolverá \[1,2,3], mientras que para la lista \[6,8,6,5,8] devolverá **false**.<br>

### forall/ 2 - El absolutismo vuelve a estar de moda.
Este es un predicado bastante sencillo, que marida muy bien con otros metapredicados, una copa de tinto y un poco de arsénico.<br>
**forall** podría verse como una especialización de include/3. De manera similar a este, verificará que para un **objetivo** determinado, todas las soluciones cumplan una condición determinada, especificada en forma de predicado. Si se cumple dicha condición **para toda solución**, devolverá **true**. Si no, fallará miserablemente:<br>

    forall(Goal, Condition).
En donde **Goal** es el término que genera soluciones, y **Condition** la condición a cumplir.<br>
    
    positivo(X) :- X > 0.
    todos_positivos(List) :-
        forall(member(X, List), positive(X)).
En este ejemplo, si todos los elementos de la lista son positivos, devuelve **true**.

## Predicados de control. Cuidado.

### once/1 - [Una y no más.](https://youtu.be/UntiFtQe1vQ?si=aF7PoC1G-d0isoZR&t=48)
Este predicado asegura que el predicado que el predicado que tiene como argumento encuentra, como mucho, **una** solución. Esto es útil para mejorar la eficiencia de un programa en puntos en los que solo necesitamos una solución y no continuar el backtracking buscando más.<br>
### \\+/1  - Inocente hasta que se demuestre lo contrario.
Habíamos hablado del **"No se puede demostrar"** hace cosa de [cuatro capítulos](../basico/Operadores.ipynb). Como refresco de la lección anterior, recordaremos que este **NO** es un predicado de negación, si no que simplemente asume que algo es falso en tanto no se demuestre lo contrario. La principal ventaja es que nos permite realizar un control **dinámico**, pudiendo responder este predicado a situaciones en las que aparezcan nuevas reglas sin intervención nuestra.<br>
### fail/0 - El caballo de Atila.
Este predicado provoca que el predicado "crece_hierba(X)." siempre devuelva un fallo allá por dónde pasa.<br>
Sencillamente, provoca un fallo automático en el momento que es llamado, es el equivalente a una contradicción lógica(Es falso por el mero hecho de existir).<br>
### true/0 - Pa ti la perra gorda.
Al contrario que **fail**, **true** siempre devuelve **true**.<br> 
**SORPRESA**.<BR>
**PLOT TWIST.**<BR>
**EMOCIÓN**.<br>
Es el equivalente a una tautología lógica(Es verdadero por el mero hecho de existir).<br>
### ! - Mención al corte se deja para el último capítulo.


## Predicados sobre la base de hechos
/dynamic

###  assert/1, asserta/1 y assertz/1 - [Solo una cosa más.](https://youtu.be/biW9BbWJtQU?si=MbeyqWG0hf3BHbq_&t=20)
Estos tres predicados son los encargados de añadir **nuevos hechos y reglas** de manera dinámica(En tiempo de ejecución) a nuestra base de conocimiento.<br>
El primero, **assert/1**, añade la nueva cláusula al **final** de la base de hechos. Tiene exactamente el mismo comportamiento que **assertz/1** pero está **DEPRECIADO** y no se aconseja seguir utilizándolo, dado que no se tiene control sobre la situación en donde se inserta la cláusula.<br>
El segundo, **asserta/1**, añade la nueva cláusula al **principio** de la base de hechos.<br>
El tercero, **assertz/1**, añade, como ya hemos dicho, la nueva cláusula al **final** de la base de hechos, y es la sintaxis que se recomienda frente a **assert/1**.
### retract/1 y retractall/1 - [Estoy cansado, jefe](https://youtu.be/_9jC1kXyYA0?si=h4E8kN6jt8di-8nT&t=83)
Estos dos predicados son el opuesto a la familia de **assert**, pero fundamentalmente tienen la misma labor aunque al revés.<br>
**"retract/1"** eliminará de forma dinámica(En tiempo de ejecución) una ocurrencia de la cláusula especificada de la base de hechos, mientras que **retractall/1** hará el equivalente a echarle gasolina al césped, prenderle fuego, coger las cenizas, mezclarlas con napalm, prenderles fuego otra vez, tirarlas en un campo, coger un avión, lanzar una bomba nuclear encima y despedirse saludando como la reina Victoria.<br>
Vamos, que revienta todas las cláusulas que puedan unificar con lo especificado. Una cosa a tener en cuenta es que el siguiente código:<br>

    retractall(predicado_importante(_)).
Es el equivalente a hacerle un drop a la base de datos de tu puesto de trabajo y te la puede liar **mal**.

### - [Siguiente capítulo - Estructuras de datos](../Avanzado/Estructuras.ipynb)
### - [Apartado anterior - **Más** variables y recursividad](Recursividad.ipynb)
### - [Volver al índice](../Indice.ipynb)